# سیشن 2 – RAG کی تشخیص راگاس کے ساتھ

راگاس میٹرکس استعمال کرتے ہوئے کم سے کم RAG پائپ لائن کی تشخیص کریں: جواب کی مطابقت، درستگی، سیاق و سباق کی درستگی۔


# منظر نامہ
یہ منظر نامہ ایک مختصر Retrieval Augmented Generation (RAG) پائپ لائن کو مقامی طور پر جانچتا ہے۔ ہم:
- ایک چھوٹا مصنوعی دستاویزاتی ذخیرہ تشکیل دیتے ہیں۔
- دستاویزات کو ایمبیڈ کرتے ہیں اور ایک سادہ مماثلت تلاش کرنے والا نافذ کرتے ہیں۔
- ایک مقامی ماڈل (Foundry Local / OpenAI-compatible) کا استعمال کرتے ہوئے مستند جوابات تیار کرتے ہیں۔
- ragas میٹرکس (`answer_relevancy`, `faithfulness`, `context_precision`) کا حساب لگاتے ہیں۔
- ایک تیز رفتار موڈ (env `RAG_FAST=1`) کی حمایت کرتے ہیں تاکہ جلدی سے جواب کی مطابقت کا حساب لگایا جا سکے۔

اس نوٹ بک کو استعمال کریں تاکہ تصدیق کی جا سکے کہ آپ کا مقامی ماڈل + ایمبیڈنگز اسٹیک بڑے ذخیرے پر جانے سے پہلے حقیقت پر مبنی جوابات پیدا کرتا ہے۔


### وضاحت: ڈیپینڈنسی انسٹالیشن
ضروری لائبریریاں انسٹال کرتا ہے:
- `foundry-local-sdk` مقامی ماڈل مینجمنٹ کے لیے۔
- `openai` کلائنٹ انٹرفیس کے لیے۔
- `sentence-transformers` گھنے ایمبیڈنگز کے لیے۔
- `ragas` + `datasets` تشخیص اور میٹرک حساب کے لیے۔
- `langchain-openai` ایڈاپٹر ragas LLM انٹرفیس کے لیے۔

دوبارہ چلانا محفوظ ہے؛ اگر ماحول پہلے سے تیار ہے تو چھوڑ دیں۔


In [1]:
# Install libraries (ragas pulls datasets, evaluate, etc.)
!pip install -q foundry-local-sdk openai sentence-transformers ragas datasets numpy langchain-openai

### وضاحت: کور امپورٹس اور میٹرکس
کور لائبریریاں اور راگس میٹرکس لوڈ کرتا ہے۔ اہم اجزاء:
- ایمبیڈنگز کے لیے SentenceTransformer۔
- `evaluate` + منتخب کردہ راگس میٹرکس۔
- تشخیصی کارپس بنانے کے لیے `Dataset`۔
یہ امپورٹس ریموٹ کالز کو متحرک نہیں کرتے (سوائے ایمبیڈنگز کے لیے ممکنہ ماڈل کیش لوڈ کے)۔


In [2]:
import os, numpy as np
from sentence_transformers import SentenceTransformer
from foundry_local import FoundryLocalManager
from openai import OpenAI
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_precision
from datasets import Dataset

### وضاحت: کھلونا کارپس اور QA گراؤنڈ ٹروتھ  
ایک چھوٹے سے ان-میموری کارپس (`DOCS`)، صارف کے سوالات کا ایک سیٹ، اور متوقع درست جوابات کی وضاحت کرتا ہے۔ یہ بیرونی ڈیٹا حاصل کیے بغیر تیز اور متعین میٹرک حساب کو ممکن بناتے ہیں۔ حقیقی حالات میں، آپ پروڈکشن سوالات اور منتخب کردہ جوابات کا نمونہ لیں گے۔


In [3]:
DOCS = [
 'Foundry Local exposes a local OpenAI-compatible endpoint.',
 'RAG retrieves relevant context snippets before generation.',
 'Local inference improves privacy and reduces latency.',
]
QUESTIONS = [
 'What advantage does local inference offer?',
 'How does RAG improve grounding?',
]
GROUND_TRUTH = [
 'It reduces latency and preserves privacy.',
 'It adds retrieved context snippets for factual grounding.',
]

### وضاحت: سروس انیشیئٹ، ایمبیڈنگز اور سیفٹی پیچ
Foundry Local مینیجر کو انیشیئلائز کرتا ہے، `promptTemplate` کے لیے اسکیمہ ڈرفٹ سیفٹی پیچ لگاتا ہے، ماڈل آئی ڈی کو ریزولو کرتا ہے، OpenAI کے ساتھ مطابقت رکھنے والا کلائنٹ بناتا ہے، اور دستاویزات کے مجموعے کے لیے ڈینس ایمبیڈنگز کو پہلے سے کمپیوٹ کرتا ہے۔ یہ بازیافت اور جنریشن کے لیے قابل استعمال حالت تیار کرتا ہے۔


In [4]:
import os
from foundry_local import FoundryLocalManager
from foundry_local.models import FoundryModelInfo
from openai import OpenAI

# --- Safe monkeypatch for potential null promptTemplate field (schema drift guard) ---
_original_from_list_response = FoundryModelInfo.from_list_response

def _safe_from_list_response(response):  # type: ignore
    try:
        if isinstance(response, dict) and response.get("promptTemplate") is None:
            response["promptTemplate"] = {}
    except Exception as e:  # pragma: no cover
        print(f"Warning normalizing promptTemplate: {e}")
    return _original_from_list_response(response)

if getattr(FoundryModelInfo.from_list_response, "__name__", "") != "_safe_from_list_response":
    FoundryModelInfo.from_list_response = staticmethod(_safe_from_list_response)  # type: ignore
# --- End monkeypatch ---

alias = os.getenv('FOUNDRY_LOCAL_ALIAS','phi-3.5-mini')
manager = FoundryLocalManager(alias)
print(f"Service running: {manager.is_service_running()} | Endpoint: {manager.endpoint}")
print('Cached models:', manager.list_cached_models())
model_info = manager.get_model_info(alias)
model_id = model_info.id
print(f"Using model id: {model_id}")

# OpenAI-compatible client
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')

from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
import numpy as np
doc_emb = embedder.encode(DOCS, convert_to_numpy=True, normalize_embeddings=True)


Service running: True | Endpoint: http://127.0.0.1:57127/v1
Cached models: [FoundryModelInfo(alias=gpt-oss-20b, id=gpt-oss-20b-cuda-gpu:1, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=9882 MB, license=apache-2.0), FoundryModelInfo(alias=phi-3.5-mini, id=Phi-3.5-mini-instruct-cuda-gpu:1, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=2181 MB, license=MIT), FoundryModelInfo(alias=phi-4-mini, id=Phi-4-mini-instruct-cuda-gpu:4, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=3686 MB, license=MIT), FoundryModelInfo(alias=qwen2.5-0.5b, id=qwen2.5-0.5b-instruct-cuda-gpu:3, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=528 MB, license=apache-2.0), FoundryModelInfo(alias=qwen2.5-7b, id=qwen2.5-7b-instruct-cuda-gpu:3, execution_provider=CUDAExecutionProvider, device_type=GPU, file_size=4843 MB, license=apache-2.0), FoundryModelInfo(alias=qwen2.5-coder-7b, id=qwen2.5-coder-7b-instruct-cuda-gpu:3, execution_p

c:\Users\leestott\AppData\Local\miniforge\envs\demo\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


### وضاحت: ریٹریور فنکشن
ایک سادہ ویکٹر مماثلت ریٹریور کو وضاحت کرتا ہے جو نارملائزڈ ایمبیڈنگز پر ڈاٹ پروڈکٹ استعمال کرتا ہے۔ ٹاپ-k ڈاکس واپس کرتا ہے (k=2 ڈیفالٹ ہے)۔ پروڈکشن میں اس کو ANN انڈیکس (FAISS، Chroma، Milvus) کے ساتھ تبدیل کریں تاکہ اسکیل اور لیٹینسی بہتر ہو۔


In [5]:
def retrieve(query, k=2):
    q = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)[0]
    sims = doc_emb @ q
    return [DOCS[i] for i in sims.argsort()[::-1][:k]]

### وضاحت: جنریشن فنکشن
`generate` ایک محدود پرامپٹ تیار کرتا ہے (سسٹم ہدایت دیتا ہے کہ صرف سیاق و سباق استعمال کریں) اور لوکل ماڈل کو کال کرتا ہے۔ کم درجہ حرارت (0.1) تخلیقی صلاحیت کے بجائے درست استخراج کو ترجیح دیتا ہے۔ مختصر جواب کا متن واپس کرتا ہے۔


In [6]:
def generate(query, contexts):
    ctx = "\n".join(contexts)
    messages = [
        {'role':'system','content':'Answer using ONLY the provided context.'},
        {'role':'user','content':f"Context:\n{ctx}\n\nQuestion: {query}"}
    ]
    resp = client.chat.completions.create(model=model_id, messages=messages, max_tokens=120, temperature=0.1)
    return resp.choices[0].message.content.strip()


### وضاحت: فال بیک کلائنٹ کی ابتدائی تشکیل
یقینی بناتا ہے کہ `client` موجود ہو، چاہے پہلے کی ابتدائی تشکیل کا سیل چھوڑ دیا گیا ہو یا ناکام ہو گیا ہو—بعد کے جائزہ مراحل کے دوران NameError کو روکتا ہے۔


In [7]:
# Fallback client initialization (added after patch failure)
try:
    client  # type: ignore
except NameError:
    from openai import OpenAI
    client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')
    print('Initialized OpenAI-compatible client (late init).')


### وضاحت: تشخیصی لوپ اور میٹرکس
تشخیصی ڈیٹا سیٹ تیار کرتا ہے (ضروری کالمز: سوال، جواب، سیاق و سباق، درست جوابات، حوالہ) اور منتخب کردہ راگاس میٹرکس پر عمل کرتا ہے۔

آپٹیمائزیشن:
- FAST_MODE صرف جواب کی مطابقت تک محدود ہے تاکہ جلدی ابتدائی ٹیسٹ کیے جا سکیں۔
- ہر میٹرک کے لیے الگ لوپ مکمل دوبارہ حساب کتاب سے بچاتا ہے جب کوئی میٹرک ناکام ہو۔

ایک dict پیدا کرتا ہے جس میں میٹرک -> اسکور (ناکامی کی صورت میں NaN) شامل ہوتا ہے۔


In [8]:
# Build evaluation dataset with required columns (including 'reference' for context_precision)
records = []
for q, gt in zip(QUESTIONS, GROUND_TRUTH):
    ctxs = retrieve(q)
    ans = generate(q, ctxs)
    records.append({
        'question': q,
        'answer': ans,
        'contexts': ctxs,
        'ground_truths': [gt],
        'reference': gt
    })

from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_precision
from langchain_openai import ChatOpenAI
from ragas.run_config import RunConfig
import math, time, os
import numpy as np

ragas_llm = ChatOpenAI(model=model_id, base_url=manager.endpoint, api_key=manager.api_key or 'not-needed', temperature=0.0, timeout=60)

class LocalEmbeddings:
    def embed_documents(self, texts):
        return embedder.encode(texts, convert_to_numpy=True, normalize_embeddings=True).tolist()
    def embed_query(self, text):
        return embedder.encode([text], convert_to_numpy=True, normalize_embeddings=True)[0].tolist()

# Fast mode: only answer_relevancy unless RAG_FAST=0
FAST_MODE = os.getenv('RAG_FAST','1') == '1'
metrics = [answer_relevancy] if FAST_MODE else [answer_relevancy, faithfulness, context_precision]

base_timeout = 45 if FAST_MODE else 120

ds = Dataset.from_list(records)
print('Evaluation dataset columns:', ds.column_names)
print('Metrics to compute:', [m.name for m in metrics])

results_dict = {}
for metric in metrics:
    t0 = time.time()
    try:
        cfg = RunConfig(timeout=base_timeout, max_workers=1)
        partial = evaluate(ds, metrics=[metric], llm=ragas_llm, embeddings=LocalEmbeddings(), run_config=cfg, show_progress=False)
        raw_val = partial[metric.name]
        if isinstance(raw_val, list):
            numeric = [v for v in raw_val if isinstance(v, (int, float))]
            score = float(np.nanmean(numeric)) if numeric else math.nan
        else:
            score = float(raw_val)
        results_dict[metric.name] = score
    except Exception as e:
        results_dict[metric.name] = math.nan
        print(f"Metric {metric.name} failed: {e}")
    finally:
        print(f"{metric.name} finished in {time.time()-t0:.1f}s -> {results_dict[metric.name]}")

print('RAG evaluation results:', results_dict)
results_dict

Evaluation dataset columns: ['question', 'answer', 'contexts', 'ground_truths', 'reference']
Metrics to compute: ['answer_relevancy']


LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


answer_relevancy finished in 78.1s -> 0.6975427764759168
RAG evaluation results: {'answer_relevancy': 0.6975427764759168}


{'answer_relevancy': 0.6975427764759168}


---

**ڈسکلیمر**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کا استعمال کرتے ہوئے ترجمہ کی گئی ہے۔ ہم درستگی کے لیے کوشش کرتے ہیں، لیکن براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا غیر درستیاں ہو سکتی ہیں۔ اصل دستاویز کو اس کی اصل زبان میں مستند ذریعہ سمجھا جانا چاہیے۔ اہم معلومات کے لیے، پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ ہم اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کے ذمہ دار نہیں ہیں۔
